**Exemplo de Chamada de Procedimento Remoto com gRPC**

Primeiro vamos instalar as dependências.


In [ ]:
!pip install grpcio grpcio-tools


Vamos criar nossa interface RPC definida em um arquivo proto chamado "my_service.proto". A seguir, uma possível definição dessa interface:

In [ ]:
%%writefile my_service.proto
syntax = "proto3";

package my_service;

service MyService {
  rpc SayHello (HelloRequest) returns (HelloResponse) {}
}

message HelloRequest {
  string name = 1;
}

message HelloResponse {
  string message = 1;
}


Overwriting my_service.proto


Em seguida, podemos usar o compilador gRPC para gerar o código Python correspondente à nossa interface RPC:

In [ ]:
!python -m grpc_tools.protoc -I./ --python_out=. --grpc_python_out=. my_service.proto


Isso gerará os arquivos my_service_pb2.py e my_service_pb2_grpc.py, que contêm as classes de mensagem e stubs de cliente/servidor para nossa interface RPC.

In [ ]:
!ls
!python my_service_pb2_grpc.py

Agora podemos criar nosso servidor gRPC em Python:

In [ ]:
import grpc
import my_service_pb2
import my_service_pb2_grpc
from  concurrent import futures

class MyService(my_service_pb2_grpc.MyServiceServicer):
    def SayHello(self, request, context):
        return my_service_pb2.HelloResponse(message=f"Hello, {request.name}!")

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    my_service_pb2_grpc.add_MyServiceServicer_to_server(MyService(), server)
    server.add_insecure_port("[::]:50051")
    server.start()
    server.wait_for_termination()

if __name__ == "__main__":
    serve()


Nesse exemplo, criamos uma classe MyService que implementa a interface RPC definida em my_service.proto. O método SayHello recebe uma requisição HelloRequest contendo um campo name e retorna uma resposta HelloResponse contendo uma mensagem de saudação.

Em seguida, criamos um servidor gRPC usando o módulo grpc do Python. Usamos a classe grpc.server para criar um novo servidor, adicionamos nossa implementação da interface RPC ao servidor usando a função add_MyServiceServicer_to_server, definimos a porta do servidor para 50051 e iniciamos o servidor.

Por fim, podemos criar um cliente gRPC em Python para chamar nosso servidor:

In [ ]:
import grpc
import my_service_pb2
import my_service_pb2_grpc

def run():
    with grpc.insecure_channel("localhost:50051") as channel:
        stub = my_service_pb2_grpc.MyServiceStub(channel)
        response = stub.SayHello(my_service_pb2.HelloRequest(name="Alice"))
        print(response.message)

if __name__ == "__main__":
    run()


---
**Atividade: Criando uma aplicação de controle remoto com gRPC**

**Objetivo**

Modificar o exemplo de cliente e servidor gRPC visto anteriormente 
para criar uma aplicação de controle remoto de uma máquina Linux. O objetivo é permitir que o cliente envie comandos em formato de string para o servidor, que irá executar esses comandos em uma máquina Linux e retornar o resultado para o cliente.

**Descrição**

A aplicação deverá ser composta por um servidor e um cliente, implementados usando gRPC e Python. O servidor deve estar configurado para receber comandos em formato de string do cliente e executá-los em uma máquina Linux. O cliente deve permitir que o usuário insira os comandos que deseja executar e exibir o resultado da execução.

**Requisitos**

* O servidor deve ser capaz de receber comandos em formato de string do cliente e executá-los em uma máquina Linux;
* O cliente deve permitir que o usuário insira os comandos que deseja executar no servidor e exibir o resultado da execução;
* O código deve ser modularizado e organizado em pacotes e módulos;
Deve ser possível executar o servidor e o cliente em máquinas diferentes;
* O servidor e o cliente devem ser implementados usando gRPC e Python.

**Dicas**

* Utilize a biblioteca **subprocess** para executar comandos **bash** no servidor;
* Utilize a biblioteca **grpc** para implementar a comunicação entre o cliente e o servidor conforme o exemplo anterior;
* Utilize a biblioteca **click** para implementar a interface de linha de comando do cliente;
* Organize o código em pacotes e módulos para facilitar a manutenção e reutilização.


**Bônus**

Implemente alguma funcionalidade adicional na aplicação, como por exemplo:

* Autenticação e autorização de usuários;
* Execução de comandos em múltiplas máquinas ao mesmo tempo;
* Implementação de um sistema de logs para armazenar as informações de execução;
* Utilização de criptografia para garantir a segurança na comunicação entre o cliente e o servidor.
